<a href="https://colab.research.google.com/github/Katyapuchkova1/Practice/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install python-pptx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 14.1 MB/s eta 0:00:00


In [ ]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from PyPDF2 import PdfReader
import codecs
from pptx import Presentation
from os import listdir
from os.path import isfile, join


def txt_from_pdf(filename_pdf):
  '''Перевод из pdf в txt и сохранение в память'''
  reader = PdfReader("/content/" + filename_pdf)
  number_of_pages = len(reader.pages)
  text = ''
  for page_num in range(number_of_pages):
    text += reader.pages[page_num].extract_text()
  with open(filename_pdf[:-3] + 'txt', 'w') as f:
    f.write(text)
  return None

#Перевод из pptx в txt формат

def txt_from_pptx(filename_pptx):
    '''Перевод из pptx в txt и сохранение в память'''
    X = Presentation("/content/" + filename_pptx)  # Presentation object created
    # Then file is opened in write mode
    ftw_data = open(filename_pptx[:-4] + 'txt', "w")
    # write text from powerpoint
    # file into .txt file
    for slide in X.slides:
        for shape in slide.shapes:
            if not shape.has_text_frame:
                continue
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    ftw_data.write(run.text)
    ftw_data.close()  # The file is closed


onlyfiles4 = ['4/' + f for f in listdir('/content/4') if isfile(join('/content/' + '4' + '/', f))]
onlyfiles3 = ['3/' + f for f in listdir('/content/3') if isfile(join('/content/' + '3' + '/', f))]
onlyfiles2 = ['2/' + f for f in listdir('/content/2') if isfile(join('/content/' + '2' + '/', f))]
onlyfiles1 = ['1/' + f for f in listdir('/content/1') if isfile(join('/content/' + '1' + '/', f))]
for f in onlyfiles2:
    txt_from_pptx(f)


letters = ['й, ц, у, к, е, н, г, ш, щ, з, х, ъ, ф, ы, в, а, п, р, о, л, д, ж, э, я, ч, с, м, и, т, ь, б, ю']
vowels = ['а', 'е', 'ы', 'у', 'э', 'о', 'я', 'и', 'ю']
punctuation = [',', ';', '.', ':', '?', ')', '(', '!']

FileNotFoundError: [Errno 2] No such file or directory: '/content/4'

In [ ]:
#Получение словаря сложных слов
tokenizer=RegexpTokenizer(r'\n\w{4,}\n')
url_diff = 'https://mydocx.ru/9-118219.html'
vocab = requests.get(url_diff)
soup = BeautifulSoup(vocab.content, 'html.parser')
text_vocab = soup.get_text()
vocab_del = tokenizer.tokenize(text_vocab.lower())
vocab = map(lambda x: x.strip('\n'), vocab_del)
vocab_dif = list(vocab)[1:]

In [ ]:
CONSTANT_SCALING = 4
#Метрики удобочитаемости

def calc_Flesh_Kincaid_rus(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid для русского языка"""
    n = 220.755 - 1.315 * (float(n_words) / n_sent) - 50.1 * (float(n_syllabes) / n_words)
    return n/CONSTANT_SCALING

def calc_Flesh_Kincaid_Grade_rus(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid Grade для русского языка"""
#    n = 0.59 * (float(n_words) / n_sent) + 6.2 * (float(n_syllabes) / n_words) - 16.59
    n = 0.49 * (float(n_words) / n_sent) + 7.3 * (float(n_syllabes) / n_words) - 16.59
    return n/CONSTANT_SCALING

FLG_X_GRADE = 0.318
FLG_Y_GRADE = 14.2
FLG_Z_GRADE = 30.5

def calc_Flesh_Kincaid_Grade_rus_flex(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid Grade для русского языка с константными параметрами"""
    if n_words == 0 or n_sent == 0: return 0
    n = FLG_X_GRADE * (float(n_words) / n_sent) + FLG_Y_GRADE * (float(n_syllabes) / n_words) - FLG_Z_GRADE
    return n/CONSTANT_SCALING

CLI_X_GRADE = 0.055
CLI_Y_GRADE = 0.35
CLI_Z_GRADE = 20.33


def calc_Coleman_Liau_index(n_letters, n_words, n_sent):
    """ Метрика Coleman Liau для русского языка с константными параметрами """
    if n_words == 0: return 0
    n = CLI_X_GRADE * (n_letters * (100.0 / n_words)) - CLI_Y_GRADE * (n_sent * (100.0 / n_words)) - CLI_Z_GRADE
    return n/CONSTANT_SCALING

SMOG_X_GRADE = 1.
SMOG_Y_GRADE = 64.6
SMOG_Z_GRADE = 0.05

def calc_SMOG_index(n_psyl, n_sent):
    """Метрика SMOG для русского языка с константными параментрами"""
    n = SMOG_X_GRADE * sqrt((float(SMOG_Y_GRADE) / n_sent) * n_psyl) + SMOG_Z_GRADE
    return n/CONSTANT_SCALING

DC_X_GRADE = 0.552
DC_Y_GRADE = 0.273

def calc_Dale_Chale_index(n_psyl, n_words, n_sent):
    """Метрика Dale Chale для русского языка с константным параметрами"""
    n = DC_X_GRADE * (100.0 * n_psyl / n_words) + DC_Y_GRADE * (float(n_words) / n_sent)
    return n/CONSTANT_SCALING

ARI_X_GRADE = 6.26
ARI_Y_GRADE = 0.2805
ARI_Z_GRADE = 31.04


def calc_ARI_index(n_letters, n_words, n_sent):
    """ Метрика Automated Readability Index (ARI) для русского языка с константными параметрами """
    if n_words == 0 or n_sent == 0: return 0
    n = ARI_X_GRADE * (float(n_letters) / n_words) + ARI_Y_GRADE * (float(n_words) / n_sent) - ARI_Z_GRADE
    return n/CONSTANT_SCALING

In [ ]:
from string import punctuation
import nltk
nltk.download('punkt')
#Подсчет характеристик для формул
def count_metrics(text):
  """Подсчет основных характеристик текста"""
  sents = sent_tokenize(text)
  num_sent = len(sents)
  num_words = 0
  num_letters = 0
  num_syl = 0
  num_hardwords = 0
  for sent in sents:
    words = word_tokenize(sent)
    words_cleaned =  list(filter(lambda x: x not in punctuation, words))
    num_words += len(words_cleaned)
    for word in words_cleaned:
      if word in vocab_dif: num_hardwords += 1
      num_letters += len(word)
      for letter in word:
        if letter in vowels: num_syl += 1
  return (num_sent, num_words, num_letters, num_syl, num_hardwords)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
